In [1]:
import cv2
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import os

In [2]:
df = pd.read_csv(r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 4 - Experimental Forest\L4V1_GT.csv")
df

,Frame,#ofPoints,ManualInput,X1,Y1,X2,Y2,X3,Y3,X4,...,X6,Y6,X7,Y7,X8,Y8,X9,Y9,X10,Y10
0,0,3,1,677,692,468,299,395,250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,0,677,692,468,299,395,250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,0,677,692,468,299,395,250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,0,676,692,467,299,395,250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3,0,676,692,467,300,394,250,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,670,4,0,444,695,479,341,609,195,692.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
671,671,4,0,447,695,482,340,612,195,694.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672,672,4,0,450,695,485,340,614,194,697.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
673,673,4,0,452,695,486,339,616,193,698.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# def getROICorners(GT_df, frame_index):
    
#     max_width = 375
#     max_y = 720 # represents floor in front
#     min_y = 150 # represents "horizon"
#     slope = max_width / (max_y-min_y)
#     point_count = GT_df["#ofPoints"][frame_index]
#     frame_points = GT_df.iloc[frame_index, 3:(3+point_count*2)]
#     Xlist = []
#     Ylist = []
    
#     # iterate through each point using the y-value
#     for i in range(1, point_count*2, 2):
#         # This linear equation serves as a base
#         #print(frame_points[i])
#         if frame_points[i] == 0 and frame_points[i-1] == 0:
#             continue
#         if frame_points[i] < min_y:
#             continue
#         width = max_width - (max_y-frame_points[i]) * slope
#         width = int(width//2)
#         if frame_points[i-1]+width > 1280:
#             Xlist.append(1275)
#         else:
#             Xlist.append(frame_points[i-1]+width)
#         Ylist.append(frame_points[i])
#         if frame_points[i-1]-width < 0:
#             Xlist.append(0)
#         else:
#             Xlist.append(frame_points[i-1]-width)
#         Ylist.append(frame_points[i])
#     return Xlist, Ylist

In [4]:
def getROICorners(GT_df, frame_index, max_width=325):
    
    #max_width = 375
    max_y = 720 # represents floor in front
    min_y = 150 # represents "horizon"
    slope = max_width / (max_y-min_y)
    point_count = GT_df["#ofPoints"][frame_index]
    frame_points = GT_df.iloc[frame_index, 3:(3+point_count*2)]
    Xlist = []
    Ylist = []
    
    # iterate through each point using the y-value
    for i in range(1, point_count*2, 2):
        # This linear equation serves as a base
        #print(frame_points[i])
        if i == 1 and frame_points[i] != 720:
            frame_points[i] = 720
        if frame_points[i] == 0 and frame_points[i-1] == 0:
            continue
        if frame_points[i] < min_y and i != 1:
            x1 = frame_points[i-3]
            y1 = frame_points[i-2]
            x2 = frame_points[i-1]
            y2 = frame_points[i]
            #print(x1, y1, x2, y2)
            if (x2-x1 == 0):
                frame_points[i-1] = x2
                frame_points[i] = min_y
            else:
                slope = (y2-y1)/(x2-x1)
                intercept = y1 - slope * x1
                new_x = (min_y-intercept) / slope
                frame_points[i-1] = new_x
                frame_points[i] = min_y
            #print("Old X value = {}, New X Value = {}".format(x2, frame_points[i-1]))
        width = max_width - (max_y-frame_points[i]) * slope
        width = int(width//2)
        if frame_points[i-1]+width > 1280:
            Xlist.append(1275)
        else:
            Xlist.append(frame_points[i-1]+width)
        Ylist.append(frame_points[i])
        if frame_points[i-1]-width < 0:
            Xlist.append(0)
        else:
            Xlist.append(frame_points[i-1]-width)
        Ylist.append(frame_points[i])
    return Xlist, Ylist

In [5]:
def draw_polygon(image, points, color):
    points = [[point[0], point[1]] for point in points]
    points = np.array([points], dtype=np.int32)
    cv2.fillPoly(image, points, color)

In [6]:
def applyMask(frame, polygon, color='green'):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    # Get the coordinates of the polygon
    roi_corners = np.array(polygon.exterior.coords, dtype=np.int32)
    cv2.fillPoly(mask, [roi_corners], (255, 255, 255))

    # Define the mask color based on the input color
    if color == 'green':
        mask_color = (0, 175, 0)
    elif color == 'blue':
        mask_color = (175, 0, 0)
    elif color == 'red':
        mask_color = (0, 0, 175)

    darkened_mask = cv2.bitwise_and(mask, mask, mask=mask)

    # Create an overlay with the same size as the image
    overlay = np.zeros_like(frame)
    overlay[darkened_mask != 0] = mask_color

    masked_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)

    output = cv2.addWeighted(frame, 1, masked_overlay, 0.25, 0.5)
    return output

In [7]:
def calculate_iou(polygon1, polygon2):
    # Clean the polygons to fix any self-intersection issues
    clean_polygon1 = polygon1.buffer(0)
    clean_polygon2 = polygon2.buffer(0)

    # Calculate the intersection and union of the cleaned polygons
    intersection_area = clean_polygon1.intersection(clean_polygon2).area
    union_area = clean_polygon1.union(clean_polygon2).area


    # Check if the union area is zero
    if union_area == 0:
        iou = 0  # Handle the case where there is no union
    else:
        # Calculate the IoU
        iou = intersection_area / union_area
        iou = intersection_area / union_area

    return iou

In [8]:
def list_files(folder_path):
    # Get a list of all files in the folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    return files

In [9]:
locationList = ["L4", "L5"]
videoFolderPath = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 4 - Experimental Forest"
GT1FolderPath = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsJimmy\\"
GT2FolderPath = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsPranava\\"
GT1FolderPath += locationList[0]
GTFiles = list_files(GT1FolderPath)
print(GTFiles)
videoName = GTFiles[0][:-7] + ".mp4"
print(videoName)

['L4V10_GT.csv', 'L4V1_GT.csv', 'L4V2_GT.csv', 'L4V3_GT.csv', 'L4V5_GT.csv', 'L4V6_GT.csv', 'L4V7_GT.csv', 'L4V8_GT.csv', 'L4V9_GT.csv']
L4V10.mp4


In [10]:
videoLocation = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 15 - Todd Falls Trail 2\L15V1.mp4"
# GT1Location = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 4 - Experimental Forest\L4V1_GT.csv"
# GT2Location = r"C:\Users\Jimmy\Downloads\L4V1_GT.csv"
# videoLocation = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 4 - Experimental Forest\L4V1.mp4"
GT1Location = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsJimmy\L15\L15V1_GT.csv"
GT2Location = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsPranava\L15\L15V1_GT.csv"

In [11]:
videoLocation = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 21 - Linwood Access Cliffside\L21V8.mp4"
# GT1Location = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 4 - Experimental Forest\L4V1_GT.csv"
# GT2Location = r"C:\Users\Jimmy\Downloads\L4V1_GT.csv"
# videoLocation = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 4 - Experimental Forest\L4V1.mp4"
GT1Location = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsJimmy\L21\L21V8_GT.csv"
GT2Location = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsPranava\L21\L21V8_GT.csv"

In [12]:
capture = cv2.VideoCapture(videoLocation)
#total_frames = capture.get(cv2.CAP_PROP_FRAME_COUNT)
# print(total_frames)
df = pd.read_csv(GT1Location)
df2 = pd.read_csv(GT2Location)
total_frames = float(df.shape[0])-1.0
print(total_frames)

827.0


In [18]:
# Single Frame

cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()

frame_number = 300
# Open the video file
cap = cv2.VideoCapture(videoLocation)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")


# Set the frame number to read
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

# Read the frame
ret, frame = cap.read()


# Check if the frame was read successfully
if not ret:
    print(f"Error: Could not read frame {frame_number}.")
    cap.release()
    
Xlist1, Ylist1 = getROICorners(df, frame_number)
Xlist2, Ylist2 = getROICorners(df2, frame_number)
points1 = []
for point in range(0, len(Xlist1), 2):
    points1.append( (Xlist1[point],Ylist1[point]) )
for point in reversed(range(1, len(Xlist1), 2)):
    points1.append( (Xlist1[point],Ylist1[point]) )
# points2 = []
# for point in range(0, len(Xlist2), 2):
#     points2.append( (Xlist2[point],Ylist2[point]) )
# for point in reversed(range(1, len(Xlist2), 2)):
#     points2.append( (Xlist2[point],Ylist2[point]) )
# polygon1 = Polygon(points1)
# polygon2 = Polygon(points2)
if len(points1) != 0:
    frame = applyMask(frame, polygon1, color='red')
# if len(points2) != 0:
#     frame = applyMask(frame, polygon2, color='blue')

# Display the frame
cv2.imshow(f"Frame {frame_number}", frame)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()

# Release the video capture object
# cap.release()

In [20]:
# Video Demo

frame_index = 0
fast_forward = 1
counter = 0
paused = False
while True:
    
    capture.set(cv2.CAP_PROP_FRAME_COUNT, frame_index)
    isTrue, frame = capture.read()
    if (frame_index < total_frames-2) & ((counter%fast_forward)==0):
        Xlist1, Ylist1 = getROICorners(df, frame_index)
        Xlist2, Ylist2 = getROICorners(df2, frame_index)
        points1 = []
        for point in range(0, len(Xlist1), 2):
            points1.append( (Xlist1[point],Ylist1[point]) )
        for point in reversed(range(1, len(Xlist1), 2)):
            points1.append( (Xlist1[point],Ylist1[point]) )
        points2 = []
        for point in range(0, len(Xlist2), 2):
            points2.append( (Xlist2[point],Ylist2[point]) )
        for point in reversed(range(1, len(Xlist2), 2)):
            points2.append( (Xlist2[point],Ylist2[point]) )
        polygon1 = Polygon(points1)
        polygon2 = Polygon(points2)
        if len(points1) != 0:
            frame = applyMask(frame, polygon1, color='red')
        if len(points2) != 0:
            frame = applyMask(frame, polygon2, color='blue')
        counter = 0
        cv2.imshow('Video', frame)
        frame_index += fast_forward
        counter+=1
    if cv2.waitKey(1) & 0xFF==ord('d'):
        cv2.destroyAllWindows()
        break
    if cv2.waitKey(1) & 0xFF==ord('f'):
        if fast_forward == 1:
            fast_forward = 5
        else:
            fast_forward = 1
    # Pause the video when 'p' key is pressed
    if cv2.waitKey(1) & 0xFF==ord('p'):
        while True:
            if cv2.waitKey(1) & 0xFF==ord('p'):
                break

            



In [ ]:
def getVideoIoU(df1, df2, total_frames,fileName):
    total_iou = 0
    frame_count = total_frames
    for frame_index in range(total_frames):
        "    #print(frame_index)\n",
        Xlist1, Ylist1 = getROICorners(df1, frame_index)
        Xlist2, Ylist2 = getROICorners(df2, frame_index)
        points1 = []
        for point in range(0, len(Xlist1), 2):
            points1.append( (Xlist1[point],Ylist1[point]) )
        for point in reversed(range(1, len(Xlist1), 2)):
            points1.append( (Xlist1[point],Ylist1[point]) )
        points2 = []
        for point in range(0, len(Xlist2), 2):
            points2.append( (Xlist2[point],Ylist2[point]) )
        for point in reversed(range(1, len(Xlist2), 2)):
            points2.append( (Xlist2[point],Ylist2[point]) )
        if len(points2)<4 or len(points1)<4:
            frame_count -= 1
            continue
        polygon1 = Polygon(points1)
        polygon2 = Polygon(points2)
        total_iou += calculate_iou(polygon1, polygon2)
    print(fileName+" IoU = ", end="")
    print(total_iou/frame_count)
    #print(frame_count)
    return total_iou/frame_count


In [ ]:
locationList = ["L4", "L5", "L6", "L7", "L8", "L9", "L10", "L11", "L12", "L13", "L14", "L15", 
                "L16", "L17", "L18", "L19", "L20", "L21","L22", "L23", "L24", "L25", "L26", 
                "L27", "L28", "L29"]
videoFolderPath = r"C:\Users\Jimmy\Documents\Experimental_Forest\VideoData720p\Location 4 - Experimental Forest"
GT1RootPath = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsJimmy\\"
GT2RootPath = r"C:\Users\Jimmy\Downloads\GroundTruthLabelsPranava\\"
GT1FolderPath = GT1RootPath + locationList[0]
GT2FolderPath = GT2RootPath + locationList[0]
GTFiles = list_files(GT1FolderPath)
#print(GTFiles)
videoName = GTFiles[0][:-7] + ".mp4"
#print(videoName)
locationIoU = 0
totalIoU = 0
for location in locationList:
    print("Analyzing "+location)
    GT1FolderPath = GT1RootPath + location
    GT2FolderPath = GT2RootPath + location
    GTFiles = list_files(GT1FolderPath)
    locationIoU = 0
    for file in range(len(GTFiles)):
        GT1Location = GT1FolderPath+"\\"+GTFiles[file]
        GT2Location = GT2FolderPath+"\\"+GTFiles[file]
        df1 = pd.read_csv(GT1Location)
        df2 = pd.read_csv(GT2Location)
        total_frames = df1.shape[0]-1
        videoName = GTFiles[file][:-7] + ".mp4"
        locationIoU += getVideoIoU(df1, df2, total_frames, videoName)
    locationIoU /= len(GTFiles)
    print("LocationIoU = ",end="")
    print(locationIoU)
    totalIoU += locationIoU
totalIoU /= len(locationList)
print("totalIoU = ",end="")
print(totalIoU)
    


In [ ]:
# L6V10 is the worst performing label at 18.85% IoU
# L15V1 is the best performing label at 89.02% IoU